In [1]:
#Packages 
import numpy as np
import cartopy
import glob
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import warnings
import xarray as xr
import xroms
from matplotlib import ticker
crs = ccrs.PlateCarree()
warnings.filterwarnings("ignore") #turns off annoying warnings
#Cartopy
land_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                edgecolor='face',
                                facecolor=cfeature.COLORS['land'])

# The core of the analysis is done with xhistogram
from xhistogram.xarray import histogram

/global/u2/d/dylan617/miconda3/envs/xroms/lib/python3.12/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [2]:
def open_roms(path):
    '''
Opens a single netcdf files with xroms. The chunks 
argument is if you get dask errors when computing gradients. 
If one appears, uncomment, then add "chunks=chunks" to 
xroms.open_netcdf. Do the same thing to open_mfnetcdf if the 
error appears!
    '''
    # chunk = {"xi": -1, "eta": -1, "ocean_time": -1} 
    # chunks = {}
    # for sub in ["rho", "u", "v", "psi"]:
    #     for k, v in chunk.items():
    #         chunks[f"{k}_{sub}"] = v
    # chunks["ocean_time"] = chunk["ocean_time"]

    ds = xroms.open_netcdf(path,)
    ds,grid = xroms.roms_dataset(ds,include_cell_volume=True)
    ds.xroms.set_grid(grid)
    return ds,grid

def open_mfroms(path):
    '''
Opens multiple netcdf files with xroms
    '''
    # chunk = {"xi": -1, "eta": -1, "ocean_time": -1} 
    # chunks = {}
    # for sub in ["rho", "u", "v", "psi"]:
        # for k, v in chunk.items():
            # chunks[f"{k}_{sub}"] = v
    # chunks["ocean_time"] = chunk["ocean_time"]

    ds = xroms.open_mfnetcdf(path)
    ds,grid = xroms.roms_dataset(ds,include_cell_volume=True)
    ds.xroms.set_grid(grid)
    return ds,grid

def add_derivatives(ds, grid, q='salt'):
    '''
Analysis of velocity gradient tensor and frontogenesis function
following Hetland et al. (2025) JPO
    '''
    qs = ds[q]
    
    #############################
    # Flow and property gradients at the ocean surface
    
    ds['dqdx'] = xroms.to_rho(grid.derivative(qs.isel(s_rho=-1), 'X'), grid)    # defined at rho-points
    ds['dqdy'] = xroms.to_rho(grid.derivative(qs.isel(s_rho=-1), 'Y'), grid)    # defined at rho-points
    
    ds['dudx'] = grid.derivative(ds.isel(s_rho=-1).u, 'X', boundary='extend')  # defined at rho-points
    ds['dvdy'] = grid.derivative(ds.isel(s_rho=-1).v, 'Y', boundary='extend')  # defined at rho-points
    ds['dvdx'] = xroms.to_rho(grid.derivative(ds.isel(s_rho=-1).v, 'X', boundary='extend'), grid)  # defined at rho-points
    ds['dudy'] = xroms.to_rho(grid.derivative(ds.isel(s_rho=-1).u, 'Y', boundary='extend'), grid)  # defined at rho-points
    
    ###########################
    # Invariant flow properties
    
    # Vorticity:  v_x - u_y
    ds['zeta'] = (ds.dvdx - ds.dudy)/ds.f
    ds['zeta'].name = 'Normalized vorticity'

    # Divergence: u_x + v_y
    ds['delta'] = (ds.dudx + ds.dvdy)/ds.f
    ds['delta'].name = 'Normalized divergence'

    # Major axis of deformation
    ds['alpha'] = ( np.sqrt( (ds.dudx-ds.dvdy)**2 + (ds.dvdx+ds.dudy)**2 ) )/ds.f
    ds['alpha'].name = 'Normalized total strain'

    ##################################
    # Principle deformation components

    ds['lminor'] = 0.5 * (ds.delta - ds.alpha)
    ds['lminor'].name = 'lambda minor'

    ds['lmajor'] = 0.5 * (ds.delta + ds.alpha)
    ds['lmajor'].name = 'lambda major'
    
    #############################################
    # Along- and cross-frontal velocity gradients
    
    # angle is wrt x, so need to do arctan2(y, x)
    ds['phi_cf'] = np.arctan2(ds.dqdy, ds.dqdx)
    ds['phi_af'] = ds.phi_cf + np.pi/2.0

    ds['du_cf'] = ( ds.dudx*np.cos(ds.phi_cf)**2 + ds.dvdy*np.sin(ds.phi_cf)**2 
               + (ds.dudy + ds.dvdx)*np.sin(ds.phi_cf)*np.cos(ds.phi_cf) )/ds.f

    ds['du_af'] = ( ds.dudx*np.cos(ds.phi_af)**2 + ds.dvdy*np.sin(ds.phi_af)**2
              + (ds.dudy + ds.dvdx)*np.sin(ds.phi_af)*np.cos(ds.phi_af) )/ds.f
    
    ############################
    # The frontogenesis function
    
    # Dimensional frontogenesis function
    Dgradq_i = - ds.dudx*ds.dqdx - ds.dvdx*ds.dqdy
    Dgradq_j = - ds.dudy*ds.dqdx - ds.dvdy*ds.dqdy
    ds['Ddelq2'] = (ds.dqdx*Dgradq_i + ds.dqdy*Dgradq_j)
    ds['Ddelq2'].name = 'Frontogenesis function'

    # Density gradients squared
    ds['gradq2'] = ds.dqdx**2 + ds.dqdy**2
    ds['gradq2'].name = r'$(\nabla q)^2$'

    # Normalized frontogenesis function
    ds['nFGF'] = 0.5 * ds.Ddelq2 / (ds.gradq2 * ds.f)
    ds['nFGF'].name = r'Normalized Frontogenesis Function'
    
    return ds

path = glob.glob('/pscratch/sd/b/bundzis/Beaufort_ROMS_2020_test_nosed_scratch/ocean_his_*.nc')
ds, grid = open_mfroms(path)
add_derivatives(ds,grid)

<xarray.Dataset> Size: 1TB
Dimensions:         (s_rho: 20, s_w: 21, ocean_time: 1481, tracer: 5,
                     boundary: 4, eta_rho: 206, xi_rho: 608, xi_u: 607,
                     eta_v: 205)
Coordinates: (12/23)
  * s_rho           (s_rho) float64 160B -0.975 -0.925 -0.875 ... -0.075 -0.025
  * s_w             (s_w) float64 168B -1.0 -0.95 -0.9 -0.85 ... -0.1 -0.05 0.0
  * ocean_time      (ocean_time) datetime64[ns] 12kB 2020-07-01T01:00:00 ... ...
    z_rho           (ocean_time, s_rho, eta_rho, xi_rho) float64 30GB dask.array<chunksize=(1, 20, 206, 608), meta=np.ndarray>
    z_w             (ocean_time, s_w, eta_rho, xi_rho) float64 31GB dask.array<chunksize=(1, 21, 206, 608), meta=np.ndarray>
    lon_rho         (eta_rho, xi_rho) float64 1MB dask.array<chunksize=(206, 608), meta=np.ndarray>
    ...              ...
    z_w_u           (ocean_time, s_w, eta_rho, xi_u) float64 31GB dask.array<chunksize=(1, 21, 206, 607), meta=np.ndarray>
    z_w_v           (ocean_time, s_w, eta_v, xi_rho) float64 31GB dask.array<chunksize=(1, 21, 205, 608), meta=np.ndarray>
    z_w_psi         (ocean_time, s_w, eta_v, xi_u) float64 31GB dask.array<chunksize=(1, 21, 205, 607), meta=np.ndarray>
    z_rho_u         (ocean_time, s_rho, eta_rho, xi_u) float64 30GB dask.array<chunksize=(1, 20, 206, 607), meta=np.ndarray>
    z_rho_v         (ocean_time, s_rho, eta_v, xi_rho) float64 30GB dask.array<chunksize=(1, 20, 205, 608), meta=np.ndarray>
    z_rho_psi       (ocean_time, s_rho, eta_v, xi_u) float64 29GB dask.array<chunksize=(1, 20, 205, 607), meta=np.ndarray>
Dimensions without coordinates: tracer, boundary
Data variables: (12/158)
    ntimes          int32 4B ...
    ndtfast         int32 4B ...
    dt              float64 8B ...
    dtfast          float64 8B ...
    dstart          datetime64[ns] 8B ...
    nHIS            int32 4B ...
    ...              ...
    phi_af          (ocean_time, eta_rho, xi_rho) float64 1GB dask.array<chunksize=(1, 206, 608), meta=np.ndarray>
    du_cf           (ocean_time, eta_rho, xi_rho) float64 1GB dask.array<chunksize=(1, 206, 608), meta=np.ndarray>
    du_af           (ocean_time, eta_rho, xi_rho) float64 1GB dask.array<chunksize=(1, 206, 608), meta=np.ndarray>
    Ddelq2          (ocean_time, eta_rho, xi_rho) float64 1GB dask.array<chunksize=(1, 206, 608), meta=np.ndarray>
    gradq2          (ocean_time, eta_rho, xi_rho) float64 1GB dask.array<chunksize=(1, 206, 608), meta=np.ndarray>
    nFGF            (ocean_time, eta_rho, xi_rho) float64 1GB dask.array<chunksize=(1, 206, 608), meta=np.ndarray>
Attributes: (12/53)
    file:              /pscratch/sd/b/bundzis/Beaufort_ROMS_2020_test_nosed_s...
    format:            netCDF-3 64bit offset file
    Conventions:       CF-1.4, SGRID-0.3
    type:              ROMS/TOMS history file
    title:             One-dimension (vertical) Sediment Toy
    var_info:          /global/homes/b/bundzis/Repos/COAWST/ROMS/External/var...
    ...                ...
    compiler_command:  /opt/cray/pe/mpich/8.1.30/ofi/gnu/12.3/bin/mpif90
    compiler_flags:    -frepack-arrays -fallow-argument-mismatch -fallow-argu...
    tiling:            032x008
    history:           ROMS/TOMS, Version 4.1, Friday - June 27, 2025 - 10:32...
    ana_file:          ROMS/Functionals/ana_btflux.h, ROMS/Functionals/ana_st...
    CPP_options:       BEAUFORT_2020_TEST_NOSED, ANA_BPFLUX, ANA_BSFLUX, ANA_...

### Construct the probability density function. 
Look for NaNs first, clean them if they exist 

In [3]:
#For a first cut, look at only a single day. 
xislice = slice(250,500)
etaslice = slice(50,130)
tslice2 = '2020-07-18'
zl = ds.zeta.sel(ocean_time=tslice2).isel(eta_rho=etaslice,xi_rho=xislice).load()

In [4]:
zetabins = np.linspace(-2,3,101)
deltabins = np.linspace(-3,2,101)
alphabins = np.linspace(0,3,101) 

zl = zl.dropna(dim='eta_rho')
zl = zl.dropna(dim='xi_rho')
# zl

In [5]:
zetapdf = histogram(zl,bins=[zetabins],density=False).load()
# Make it a probability density function by dividing by max values
zetapdf = zetapdf/zetapdf.max()
zetapdf

<xarray.DataArray 'histogram_zeta' (zeta_bin: 100)> Size: 800B
array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.89417442e-05, 6.89417442e-05,
       6.89417442e-05, 3.44708721e-04, 7.58359186e-04, 7.58359186e-04,
       1.65460186e-03, 3.37814547e-03, 4.89486384e-03, 6.68734919e-03,
       8.06618407e-03, 1.36504654e-02, 1.87521544e-02, 2.75766977e-02,
       3.90210272e-02, 5.17063082e-02, 7.74215788e-02, 1.23198897e-01,
       1.82075147e-01, 2.64943123e-01, 3.66080662e-01, 4.71492589e-01,
       5.81937263e-01, 7.00172354e-01, 8.17580145e-01, 9.23750431e-01,
       9.91106515e-01, 1.00000000e+00, 9.53809031e-01, 8.99413995e-01,
       8.04067563e-01, 6.99345053e-01, 6.08410893e-01, 5.19751810e-01,
       4.55015512e-01, 3.77593933e-01, 3.21957946e-01, 2.64322647e-01,
       2.13650465e-01, 1.88969321e-01, 1.61875215e-01, 1.32850741e-01,
       1.21751120e-01, 1.06101344e-01, 9.13478111e-02, 7.85246467e-02,
       6.69424336e-02, 5.62564633e-02, 4.90865219e-02, 4.64667356e-02,
       4.06756291e-02, 3.84694933e-02, 3.37814547e-02, 2.70941055e-02,
       2.57842123e-02, 2.44743192e-02, 2.32333678e-02, 2.00620476e-02,
       1.66839021e-02, 1.44777663e-02, 1.35815236e-02, 1.14443295e-02,
       1.07549121e-02, 9.16925198e-03, 8.13512582e-03, 7.44570838e-03,
       5.99793175e-03, 5.37745605e-03, 5.23957256e-03, 4.89486384e-03,
       3.30920372e-03, 3.58497070e-03, 3.24026198e-03, 2.41296105e-03,
       2.20613582e-03, 1.58566012e-03, 1.37883488e-03, 1.58566012e-03,
       1.10306791e-03, 7.58359186e-04, 7.58359186e-04, 8.96242675e-04,
       4.82592210e-04, 7.58359186e-04, 3.44708721e-04, 2.06825233e-04,
       2.75766977e-04, 4.13650465e-04, 3.44708721e-04, 2.06825233e-04])
Coordinates:
  * zeta_bin  (zeta_bin) float64 800B -1.975 -1.925 -1.875 ... 2.875 2.925 2.975

### This works, let's write a function to do this for any variable

In [6]:
def remove_nans(var):
    var = var.dropna(dim='eta_rho')
    var = var.dropna(dim='xi_rho')
    return var

### Make plots for vorticity, divergence, and strain 

In [7]:
deltapdf = histogram(remove_nans(ds.delta.sel(ocean_time=tslice2).isel(eta_rho=etaslice,xi_rho=xislice)),bins=[deltabins],density=False).load()
deltapdf = deltapdf/deltapdf.max()
alphapdf = histogram(remove_nans(ds.alpha.sel(ocean_time=tslice2).isel(eta_rho=etaslice,xi_rho=xislice)),bins=[alphabins],density=False).load()
alphapdf = alphapdf/alphapdf.max()
# zetaalphapdf = histogram(remove_nans(ds.alpha.sel(ocean_time=tslice2),density=True).load()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(10,2.7),dpi=100,
                      constrained_layout=True,sharey=True)

ax[0].semilogy(zetapdf.zeta_bin, zetapdf,c='k')
ax[1].semilogy(deltapdf.delta_bin, deltapdf,c='k')
ax[2].semilogy(alphapdf.alpha_bin, alphapdf,c='k')

for i in range(3):
    ax[i].grid()
    ax[i].set_ylim(10**-3,1.2) #Cut off below 1 in every 1000 points

ax[0].set_ylabel('Probability density function')
ax[0].set_xlabel(r'$(\partial_x v - \partial_y u)/f$')
ax[1].set_xlabel(r'$(\partial_x u + \partial_y v)/f$')
ax[2].set_xlabel(r'$\sqrt{(\partial_x u - \partial_y v)^2 + (\partial_x v + \partial_y u)^2}/f$')
ax[0].set_title(r'Relative vorticity')
ax[1].set_title(r'Divergence')
ax[2].set_title(r'Total strain rate')

Text(0.5, 1.0, 'Total strain rate')

### The model is definetely submesoscale permitting. Make one more figure following https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/grl.50919

In [ ]:
zetaalpha = histogram(remove_nans(ds.zeta.sel(ocean_time=tslice2).isel(eta_rho=etaslice,xi_rho=xislice)),
                      remove_nans(ds.alpha.sel(ocean_time=tslice2).isel(eta_rho=etaslice,xi_rho=xislice)),
                      bins=[zetabins,alphabins],
                      density=False).load()
zetaalpha = zetaalpha/zetaalpha.max()

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(6,4),
                    constrained_layout=True,dpi=200)

c = np.log10(zetaalpha.T).plot(vmin=-3,vmax=0,cmap=cmo.amp, add_colorbar=False)
fig.colorbar(c, extend = 'min', label = 'JPDF')
ax.grid(alpha=0.7)
ax.set_xlabel(r'$\zeta/f$')
ax.set_ylabel(r'$\alpha/f$')
ax.set_title(r'Vorticity-strain JPDF')